In [2]:
from flask import Flask, render_template, request, redirect, url_for
import random
from faker import Faker
from datetime import datetime
from jinja2 import DictLoader

app = Flask(__name__)
fake = Faker()

# Generate fake data
properties = [{
    "id": i,
    "address": fake.street_address(),
    "city": fake.city(),
    "roof_type": random.choice(["Tile", "Shingle", "Metal"]),
    "last_roof_date": fake.date_between(start_date='-30y', end_date='today').strftime('%Y-%m-%d'),
    "owner": fake.name(),
    "parcel_name": fake.company(),
    "llc_mailing_address": fake.address(),
    "contact_info": [],
    "notes": ""
} for i in range(1, 51)]

# HTML Templates in DictLoader
app.jinja_loader = DictLoader({
    'index.html': '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>SCI Roofing Permits</title>
        <script>
            function toggleDateInputs() {
                var filter = document.getElementById("date_filter").value;
                document.getElementById("date_to").style.display = filter === "between" ? "inline" : "none";
            }
        </script>
    </head>
    <body onload="toggleDateInputs()">
        <img src="{{ url_for('static', filename='SCILOGO.png') }}" style="width:150px;">
        <h2>SCI Roofing Permits</h2>
        <form method="get">
            Address/City: <input type="text" name="address" placeholder="Search Address or City" value="{{request.args.get('address', '')}}">
            Roof Type: <input type="text" name="roof_type" placeholder="Search Roof Type" value="{{request.args.get('roof_type', '')}}">
            Owner: <input type="text" name="owner" placeholder="Search Owner" value="{{request.args.get('owner', '')}}">
            Date Filter:
            <select name="date_filter" id="date_filter" onchange="toggleDateInputs()">
                <option value="">None</option>
                <option value="before" {% if request.args.get('date_filter') == 'before' %}selected{% endif %}>Before</option>
                <option value="after" {% if request.args.get('date_filter') == 'after' %}selected{% endif %}>After</option>
                <option value="between" {% if request.args.get('date_filter') == 'between' %}selected{% endif %}>Between</option>
            </select>
            <input type="date" name="date_from" value="{{request.args.get('date_from', '')}}">
            <input type="date" name="date_to" id="date_to" value="{{request.args.get('date_to', '')}}" style="display:none;">
            <button type="submit">Search</button>
        </form>
        <table border="1">
            <tr>
                <th>Property Address</th>
                <th>City</th>
                <th>Roof Type</th>
                <th>Date of Last Roof Permit</th>
                <th>Owner</th>
            </tr>
            {% for prop in properties %}
            <tr onclick="window.location.href='{{ url_for('edit_property', prop_id=prop.id) }}'">
                <td>{{prop.address}}</td>
                <td>{{prop.city}}</td>
                <td>{{prop.roof_type}}</td>
                <td>{{prop.last_roof_date}}</td>
                <td>{{prop.owner}}</td>
            </tr>
            {% endfor %}
        </table>
    </body>
    </html>
    ''',
    'edit_property.html': '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Edit Property</title>
    </head>
    <body>
        <h2>Edit Property Details</h2>
        <form method="post">
            Address: <input type="text" name="address" value="{{prop.address}}"><br>
            Roof Type: <input type="text" name="roof_type" value="{{prop.roof_type}}"><br>
            Last Roof Date: <input type="date" name="last_roof_date" value="{{prop.last_roof_date}}"><br>
            Owner: <input type="text" name="owner" value="{{prop.owner}}"><br>
            Parcel Name: <input type="text" name="parcel_name" value="{{prop.parcel_name}}"><br>
            LLC Mailing Address: <input type="text" name="llc_mailing_address" value="{{prop.llc_mailing_address}}"><br>
            Notes:<br><textarea name="notes">{{prop.notes}}</textarea><br>

            Contact Info:<br>
            <div id="contacts">
                {% for c in prop.contact_info %}
                    Email: <input type="email" name="email" value="{{c.email}}">
                    Phone: <input type="text" name="phone" value="{{c.phone}}"><br>
                {% endfor %}
                Email: <input type="email" name="email">
                Phone: <input type="text" name="phone"><br>
            </div>

            <button type="submit">Save</button>
        </form>
    </body>
    </html>
    '''
})

@app.route('/', methods=['GET', 'POST'])
def index():
    address = request.args.get('address', '').lower()
    roof_type = request.args.get('roof_type', '').lower()
    owner = request.args.get('owner', '').lower()
    date_filter = request.args.get('date_filter', '')
    date_from = request.args.get('date_from', '')
    date_to = request.args.get('date_to', '')

    filtered_properties = [prop for prop in properties if
                           address in prop['address'].lower() or address in prop['city'].lower()]

    if roof_type:
        filtered_properties = [prop for prop in filtered_properties if roof_type in prop['roof_type'].lower()]
    if owner:
        filtered_properties = [prop for prop in filtered_properties if owner in prop['owner'].lower()]

    if date_filter and date_from:
        date_from_obj = datetime.strptime(date_from, '%Y-%m-%d')
        if date_filter == 'before':
            filtered_properties = [prop for prop in filtered_properties if datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') < date_from_obj]
        elif date_filter == 'after':
            filtered_properties = [prop for prop in filtered_properties if datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') > date_from_obj]
        elif date_filter == 'between' and date_to:
            date_to_obj = datetime.strptime(date_to, '%Y-%m-%d')
            filtered_properties = [prop for prop in filtered_properties if date_from_obj <= datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') <= date_to_obj]

    return render_template('index.html', properties=filtered_properties)

@app.route('/property/<int:prop_id>', methods=['GET', 'POST'])
def edit_property(prop_id):
    prop = next((item for item in properties if item['id'] == prop_id), None)
    if request.method == 'POST':
        prop['address'] = request.form['address']
        prop['roof_type'] = request.form['roof_type']
        prop['last_roof_date'] = request.form['last_roof_date']
        prop['owner'] = request.form['owner']
        prop['parcel_name'] = request.form['parcel_name']
        prop['llc_mailing_address'] = request.form['llc_mailing_address']
        prop['notes'] = request.form['notes']

        return redirect(url_for('index'))

    return render_template('edit_property.html', prop=prop)

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Aug/2025 15:58:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2025 15:58:36] "GET /static/SCILOGO.png HTTP/1.1" 404 -
